In [5]:
# import pandas as pd
# import os
# from tqdm import tqdm
# content_list = []
# labels_list = []
# root_dir = 'raw_data'
# for category in tqdm(os.listdir(root_dir)):
#     for i in os.listdir(f"{root_dir}/{category}"):
#         path = f"{root_dir}/{category}/{i}"
#         content = open(path, 'r').read()[:10]
#         content_list.append(content)
#         labels_list.append(category)
# ans = pd.DataFrame(labels_list, content_list).reset_index()
# ans.columns = ['text', 'label']
# ans.to_csv('raw_data/total.parquet')

In [6]:
import pandas as pd
import random
df = pd.read_parquet('raw_data/total.parquet')
levels = [i for i in df.columns if i!='text']
df_group = df.groupby('label').agg(lambda x: random.sample(x.tolist(), 2000))
df = df_group.explode('text').reset_index()
df['text'] = df['text'].apply(lambda x: x.replace('\n', ''))
df = df[df['text'].apply(type) == type('')]

In [7]:
import random
import copy
test_df = copy.deepcopy(df_group)
test_df['text'] = test_df['text'].apply(lambda x: random.sample(x, len(x) // 10 * 2))
test_df = test_df.explode('text').reset_index()
test_df = test_df[['text', 'label']]
test_df.to_csv('origin_data/test.tsv', index=None, sep='\t')
test_df

,text,label
0,图文-CLPGA上海精英赛开幕 柯胜恩致辞\n 新浪体育讯 北京时间3月31日消息：201...,体育
1,西班牙踩老冤家锁定8强 25分屠杀!两年的心结解了\n 新浪体育讯 北京时间9月9日，男篮...,体育
2,曼城3800万镑豪购宣布在即 中场名将难忍低薪要离队\n 新浪体育讯 据《天空体育》、《镜...,体育
3,德塞战主裁发黄牌像发扑克牌 他的手段简单而又粗暴\n 德国对阵塞尔维亚，当值主裁判马伦科用...,体育
4,白浪选择与母亲同号球衣 郎平建议多积累主攻经验\n 新浪体育讯 为期三天的中美青年女排对抗...,体育
...,...,...
5595,短信诈骗又出新花样 切勿轻信银行卡扣年费\n 最近，一些郑州市民收到了“银行卡要被扣120...,财经
5596,基金认购向大公司好指数集中\n 本报讯 记者从银行渠道获悉，正在热销的易方达深100ETF...,财经
5597,基金整体净申购 投资者借道入市意愿增强\n ⊙记者 孙旭 ○编辑 张亦文\n 一季度渐趋...,财经
5598,规避风险 债券基金再掀分红热潮\n □本报记者 徐国杰 上海报道\n 据Wind统计显示...,财经


In [8]:
test_df = pd.read_csv('origin_data/test.tsv', sep='\t')
df = df[~df['text'].isin(test_df['text'])]
df_group = df.groupby(levels).agg(list)
df_group['len'] = df_group['text'].apply(len)
eval_df = copy.deepcopy(df_group)
eval_df['text'] = eval_df['text'].apply(lambda x: random.sample(x, len(x) // 10))
eval_df = eval_df.explode('text').reset_index()[levels + ['text']]
eval_df = eval_df[['text', 'label']]
eval_df.to_csv('origin_data/eval.tsv', index=None, sep='\t')
eval_df

,text,label
0,大师赛资格赛中国两将晋级 达赫迪负小怀特止步首轮 新浪体育讯 北京时间10月27日凌晨，0...,体育
1,CUBA个人单场得分最高纪录刷新 山西后卫疯砍51分 本报讯(首席记者李清伟)时隔一年，两...,体育
2,米查罗布赛珊珊平赛季最低杆 奥查娅扩大领先优势 新浪体育讯 北京时间5月9日，LPGA米查...,体育
3,市长澄清耗资千万亚运建筑被强拆：违法的肯定要拆 信息时报讯 据媒体报道，2月14日和17日...,体育
4,徐根宝力挺爱徒高洪波 预测国足打叙利亚状态达顶峰 本报讯 (记者郑小龙)“小高(高洪波)的...,体育
...,...,...
2795,多空无心恋战 股指期货底部或已来临 文/本刊记者郭娴洁 提要：目前多空双方都是意兴阑珊，...,财经
2796,专家：期货市场迎来创新之年 2009年中国成为全球最大的商品期货市场，而2010年将成为中...,财经
2797,AIG高管同意退回半数奖金 旗下公司更名为AIU ⊙本报记者 朱贤佳 实习记者 王宙洁 ...,财经
2798,创业板知识纳入基金经理证券投资法律知识考纲 ⊙记者 马婧妤 中国证券业协会昨日发布公告，...,财经


In [9]:
test_df = pd.read_csv('origin_data/test.tsv', sep='\t')
eval_df = pd.read_csv('origin_data/eval.tsv', sep='\t')
train_df = df[~(df['text'].isin(eval_df['text']) | df['text'].isin(test_df['text']))]
train_df = train_df[['text', 'label']]
train_df.to_csv('origin_data/train.tsv', index=None, sep='\t')
train_df

,text,label
0,周继红首次证实郭晶晶无缘伦敦奥运 已交退役申请 新浪体育讯 北京时间1月22日消息，中国跳...,体育
1,图文-F1中国站维泰尔杆位 这个庆祝姿势像舒米 新浪体育讯 北京时间4月18日下午，200...,体育
2,生死时刻杜兰特一击致命 别再怀疑他就是未来之王 俗话说人比人得死货比货得扔，真是一点不错。...,体育
3,国米清洗狂潮开始：1强人引14队争夺 两边缘人竟欲留 新浪体育讯 在国际米兰忙于为满足穆里...,体育
4,卡洛斯曝王珀制造1比5：20天前已定 把我关在酒店 记者张卫北京报道6年前，因为王珀，卡洛...,体育
...,...,...
27995,损失惨重 QDII短期宜投抗风险能力较强市场 2008年QDII(合格境内机构投资者)基金...,财经
27996,伪造交通事故责任认定书 安邦保险员工骗保23万 本报讯(记者刘杰)保险公司两名理赔员，伪造...,财经
27997,螺纹钢成本上涨 有望继续走高 □杨小强 自元旦以来，国内金属市场走势分化，有色金属前期强...,财经
27998,二季度百只股基遭弃 两股基跌破清盘警戒线 每经记者李娜发自成都 侵蚀股基规模和资产净值的...,财经
